In [47]:
from modules.result_loader import RelevantDataProcessor, RelevantDataStatistics
from modules.default_config import get_default_config
import numpy as np

In [48]:
config = get_default_config()
PROCESSED_RESULTS_FOLDER = "./results"
processor = RelevantDataProcessor(PROCESSED_RESULTS_FOLDER, config.experiments)
#processor.extract_and_save_relevant_data("graphdata.npy") # Reads 15 GB of data

### relevant data dict:
### all_stats:
###     (centroid name, function name, dimensionality)
###         count_at_min_value
###         total_min_value
###         avg_count
###         avg_min_value
###         best_vals_avg
###         best_vals_std


Config prepared, n_experiments=2550


In [49]:
data = processor.load_prepared_relevant_data("graphdata.npy")
stats = RelevantDataStatistics(data)
#stats.generate_convergence_curves_for_fc_x_dim(save="./results/graphics/convergence")

In [50]:
from modules.centroids import ALL_CENTROIDS
def normalized_ranking(data, fc_weights=None, dim_weights=None, evalfc=None):
    if fc_weights is None:
        fc_weights = {x: 1 for x in data.keys()}
    if dim_weights is None:
        dim_weights = {10: 1, 50: 1, 100: 1}

    results = {x.__qualname__: 0 for x in ALL_CENTROIDS}
    step = 1 / (len(ALL_CENTROIDS) - 1)
    for f in data.keys():
        for dim in data[f].keys():
            i = 0
            for x in data[f][dim]:
                results[x[1]] += fc_weights[f] * dim_weights[dim] * i
                i += step
    return results

In [51]:
s = stats.get_centroids_for_each_function_sorted_by("avg_min_value")

In [52]:
normalized_ranking(s, dim_weights={10: 1, 50: 0, 100: 0})

{'vanila_centroid': 12.25,
 'mean_centroid': 17.25,
 'median_centroid': 25.0,
 'interquartile_centroid': 15.25,
 'windsor_centroid': 15.25}

In [78]:
import pandas as pd
fun = pd.DataFrame({'fun_name': ['Init'], 'vanila_centroid': [0], 'mean_centroid': [0], 'median_centroid': [0], 'interquartile_centroid': [0], 'windsor_centroid': [0]})

In [79]:
for f in s.keys():
    x = {fc : 0 for fc in s.keys()}
    x[f] = 1
    ranking = normalized_ranking(s, fc_weights=x, dim_weights={10: 1, 50: 1, 100: 1})
    ranking['fun_name'] = f
    fun = fun._append(ranking, ignore_index=True)

In [81]:
fun.to_csv('fc_rank.csv')

In [83]:
dim = pd.DataFrame({'dimension': [0], 'vanila_centroid': [0], 'mean_centroid': [0], 'median_centroid': [0], 'interquartile_centroid': [0], 'windsor_centroid': [0]})

In [84]:
for d in [10, 50, 100]:
    x = {fc : 0 for fc in [10, 50, 100]}
    x[d] = 1
    ranking = normalized_ranking(s, dim_weights=x)
    ranking['dimension'] = d
    dim = dim._append(ranking, ignore_index=True)

In [86]:
dim.to_csv('dim_rank.csv')